- 군집화

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import cluster
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
my_path = os.getcwd()
concat_path = os.path.join(my_path, '어린이집병합')

KID_2020 = pd.read_csv(os.path.join(concat_path, '어린이집2020.csv'), encoding='utf-8-sig', index_col=0)
KID_2021 = pd.read_csv(os.path.join(concat_path, '어린이집2021.csv'), encoding='utf-8-sig', index_col=0)
KID_2022 = pd.read_csv(os.path.join(concat_path, '어린이집2022.csv'), encoding='utf-8-sig', index_col=0)
KID_2023 = pd.read_csv(os.path.join(concat_path, '어린이집2023.csv'), encoding='utf-8-sig', index_col=0)
KID_2024 = pd.read_csv(os.path.join(concat_path, '어린이집2024.csv'), encoding='utf-8-sig', index_col=0)

CHO_2021 = pd.read_csv(os.path.join(concat_path, '초등학교2021.csv'), encoding='utf-8-sig', index_col=0)
CHO_2022 = pd.read_csv(os.path.join(concat_path, '초등학교2022.csv'), encoding='utf-8-sig', index_col=0)
CHO_2023 = pd.read_csv(os.path.join(concat_path, '초등학교2023.csv'), encoding='utf-8-sig', index_col=0)

In [ ]:
Real_Kmeans = cluster.KMeans(n_clusters=len(center_point))
Real_Kmeans.cluster_centers_ = np.array(center_point, order='C')
Real_Kmeans.fit(kid_point)
centers = Real_Kmeans.cluster_centers_
pred = Real_Kmeans.predict(kid_point)
kid_point['pred'] = pred

plt.figure(figsize=(12,12))
sns.scatterplot(x=kid_point['위도'], y=kid_point['경도'], hue=kid_point['pred'])
# plt.scatter(centers[:,0], centers[:,1])

In [ ]:
# 1개의 군집마다 파악하기
number = 5

plt.figure(figsize=(20,20))
sns.scatterplot(x=kid_point['위도'], y=kid_point['경도'], hue=(kid_point['pred'] == number))
plt.scatter(centers[number,0], centers[number,1], )

In [ ]:
# 1개의 군집마다 파악하기
number = 1
condition = (kid_point['pred'] == number)

plt.figure(figsize=(10,10))
sns.scatterplot(x=kid_point[condition]['위도'], y=kid_point[condition]['경도'])
plt.scatter(centers[number,0], centers[number,1], )

In [ ]:
def get_index(x:int, df, center):
    return center[(center['위도'] == df.loc[x, '위도']) & (center['경도'] == df.loc[x, '경도'])].index[0]

In [ ]:
df['pred'] = [get_index(i, df, center_point) for i in df.index]

In [ ]:
sido_db = pd.DataFrame({'시도': list(KID_2020.groupby(['시도'])['어린이집명'].count().index), '개수':list(KID_2020.groupby(['시도'])['어린이집명'].count().values)})
sido_db

In [ ]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium import Marker

map = folium.Map()

marker_cluster = MarkerCluster().add_to(map)

# 초등학교 찍기
for j in df.index:
    name = df.loc[j, '학교명']
    address = df.loc[j, '학교도로명 주소']
    tooltip_cho = name + "(" + address + ")"
    location_cho = [df.loc[j, '위도'], df.loc[j, '경도']]
    icon = folium.Icon(color='red', icon='star')
    folium.Marker(location=location_cho, tooltip=tooltip_cho, icon=icon).add_to(marker_cluster)

# 유치원 찍기
mc = MarkerCluster()
for n in kid_point.index:
    pred_index = kid_point.loc[n, "pred"]
    try :
        cho_name = df.loc[df[df['pred'] == pred_index].index[0], '학교명']
        kid_name = KID_2020.loc[KID_2020[(KID_2020['위도'] == kid_point.loc[n, '위도']) & (KID_2020['경도'] == kid_point.loc[n, '경도'])].index[0], '어린이집명']
        kid_count = KID_2020.loc[KID_2020[(KID_2020['위도'] == kid_point.loc[n, '위도']) & (KID_2020['경도'] == kid_point.loc[n, '경도'])].index[0], '현원수']
    except:
        cho_name = 'X'
        kid_name = 'X'
        kid_count = 0
    
    tooltip = cho_name + " / " + kid_name + "(" + str(kid_count) + ")"
    location = [kid_point.loc[n, "위도"], kid_point.loc[n, "경도"]]
    mc.add_child(folium.Marker(location = location, tooltip=tooltip))

map.add_child(mc)

with open('Si_Do_map_utf8.json', 'r', encoding='euc-kr') as file:
    Sido_geo = json.load(file)

#밀도 및 경계선 그리기
folium.Choropleth(
    geo_data = Sido_geo, #경계선
    data = sido_db, #데이터
    columns=["시도", "개수"], #데이터
    key_on = 'feature.properties.CTP_KOR_NM', #경계선
    fill_color="PuRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="어린이집").add_to(map)

map

In [ ]:
map.save('배정학교.html')